In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ECz
from tqdm import tqdm
import time
import pandas as pd

from bs4 import BeautifulSoup as bs

In [4]:
# 웹드라이버 실행
driver = webdriver.Chrome()
# 창을 최대로 키우기
driver.maximize_window()

# 제품 목록 페이지 로드
url = 'https://www.lg.com/in/tv-soundbars/all-tv-soundbars/?ec_model_status_code=ACTIVE&firstResult=12&sortCriteria=%40ec_spotlights_order_no+descending%2C%40ec_salable_status+descending%2C%40ec_pdp_view_count+descending'

# 페이지 로딩 대기
time.sleep(3)

# 정적 스크랩핑으로 raw 웹 데이터 저장
soup = bs(driver.page_source, 'lxml')
links = soup.select('ul div.c-product-item__ufn a')

# 모든 제품 페이지에서 제품 링크 수집
product_links = []

for link in links:
    product_links.append('https://www.lg.com/'+link['href'])

# 웹드라이버 실행
driver = webdriver.Chrome()
# 창을 최대로 키우기
driver.maximize_window()

for i in tqdm(range(len(product_links))):
    
    driver.get(product_links[i])
    
    # 리뷰 탭 클릭
    review_tab = driver.find_elements(By.CLASS_NAME, 'c-tabs__item-text')
    review_tab[3].click()
    time.sleep(3)  # 리뷰 탭 로딩 대기

    # Shadow DOM의 root 접근 함수 선언
    def shadow_root(element):
        shadow_root = driver.execute_script('return arguments[0].shadowRoot',element)
        return shadow_root

    # 리뷰 데이터가 있는 섹션 찾기
    shadow_data = driver.find_element(By.CSS_SELECTOR, '[data-bv-show="reviews"]')
    shadow_data_open = shadow_root(shadow_data)

    # 스크롤 내려서 Locale 필터 설정
    # element = shadow_data_open.find_element(By.CSS_SELECTOR, '#bv_review_maincontainer > div.bv -rnr__sc-16dr7i1-0.bv-rnr__sc-16dr7i1-22.hiYLYm.gapRtl')
    # element = shadow_data_open.find_element(By.CLASS_NAME, 'bv-rnr__sdlnh3-0.iFkpgv')
    element = shadow_data_open.find_element(By.CLASS_NAME, 'bv-rnr__sc-1wzl89g-0.gpRfFu')
    # element = shadow_data_open.find_element(By.CLASS_NAME, 'bv-rnr__sc-13ijx62-3.fVALEs')
    
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", element)
    time.sleep(2)  # 스크롤 완료 대기
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", element)
    time.sleep(2)  # 스크롤 완료 대기

    # Locale filter
    locale_filter = shadow_data_open.find_element(By.CSS_SELECTOR, '[aria-controls="bv-reviews-filter-Locale"]')
    # locale_filter = shadow_data_open.find_element(By.CSS_SELECTOR, '#bv_review_maincontainer > div.bv-rnr__sc-16dr7i1-0.bv-rnr__sc-16dr7i1-22.hiYLYm.gapRtl > div > div:nth-child(4) > div')
    time.sleep(3)
    locale_filter.click()
    time.sleep(2)
    locale_filter.click()
    english_india = shadow_data_open.find_elements(By.CLASS_NAME, 'bv-rnr__sc-1tvt4yi-5.kveQCZ')
    english_india[1].click()
    time.sleep(2)
    
    # Lowest to Highest Rating
    low_to_high = shadow_data_open.find_element(By.CSS_SELECTOR, '#bv_review_maincontainer > div.bv-rnr__sc-11r39gb-1.ewPPY > div > div.bv-rnr__wnep33-0.cIbVOf > div > div.bv-rnr__sc-13ijx62-0.bmAjQc > div > div')
    time.sleep(2)
    low_to_high.click()
    time.sleep(1)
    low_to_high.click()
    low_to_high_select = shadow_data_open.find_element(By.ID, 'bv-reviews-filters-negative-3')
    low_to_high_select.click()
    time.sleep(1)

    # 리뷰 컨테이너 1
    users = shadow_data_open.find_elements(By.CLASS_NAME, 'bv-rnr__emkap-0.eyhvsJ')
    user_list = []
    for user in users:
        user_list.append(user.text)

    # 리뷰 컨테이너 2-1
    review_a_list = []
    try:
        reviews_a = shadow_data_open.find_elements(By.CLASS_NAME, 'bv-rnr__sc-16dr7i1-10.eIsLQk')
        for review_a in reviews_a:
            review_a_list.append(review_a.text)
    except:
        pass
    
    # 리뷰 컨테이너 2-2
    review_b_list = []
    try:
        reviews_b = shadow_data_open.find_elements(By.CLASS_NAME, 'bv-rnr__sc-16dr7i1-10.hDttlK')
        for review_b in reviews_b:
            review_b_list.append(review_b.text)
    except:
        pass
    review_a_b_list = review_a_list + review_b_list

0it [00:00, ?it/s]


In [9]:
# 크롬 드라이버 경로 설정
# service = Service('/path/to/chromedriver')  # 크롬 드라이버 경로를 설정하세요.

In [12]:
# 각 제품군에 대한 URL 정의
urls = {
    "TV": "https://www.lg.com/in/tv-soundbars/all-tv-soundbars/?ec_model_status_code=ACTIVE",
    "세탁기": "https://www.lg.com/in/laundry/all-washing-machines/?ec_model_status_code=ACTIVE",
    "에어컨": "https://www.lg.com/in/air-conditioners/all-air-conditioners/?ec_model_status_code=ACTIVE",
    "냉장고": "https://www.lg.com/in/refrigerators/all-refrigerators/?ec_model_status_code=ACTIVE"
}

In [13]:
# 리뷰 데이터를 저장할 딕셔너리
reviews = {}

# 브라우저 시작
driver = webdriver.Chrome() #options=options
wait = WebDriverWait(driver, 5)

In [ ]:
# 각 제품군에서 데이터를 스크래핑하는 함수
def scrape_reviews(url, category):
    driver.get(url)

    # 'Most Popular'로 정렬
    sort_by_dropdown = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'select[name="sort"]')))
    sort_by_dropdown.click()
    most_popular_option = driver.find_element(By.XPATH, '//option[@value="MOST_POPULAR"]')
    most_popular_option.click()

    while True:
        product_links = driver.find_elements(By.CSS_SELECTOR, 'div.model-list > div.model-item > a')

        for link in tqdm(product_links, desc=f"Processing {category} products"):
            product_url = link.get_attribute('href')
            driver.get(product_url)
            
            # 리뷰 탭 클릭
            reviews_tab = wait.until(EC.element_to_be_clickable((By.XPATH, '//a[contains(text(), "Reviews")]')))
            reviews_tab.click()

            # 'Locale'을 'English (India)'로 설정
            locale_dropdown = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'select#reviewFilterLocale')))
            locale_dropdown.click()
            english_india_option = driver.find_element(By.XPATH, '//option[contains(text(), "English (India)")]')
            english_india_option.click()

            # 'Sort by'를 'Lowest to Highest Rating'으로 설정
            sort_dropdown = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'select#reviewSortOrder')))
            sort_dropdown.click()
            lowest_to_highest_option = driver.find_element(By.XPATH, '//option[@value="rating-asc"]')
            lowest_to_highest_option.click()

            # 리뷰 요소 찾기
            review_elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.cmmnt-contents')))
            if category not in reviews:
                reviews[category] = []
            reviews[category].extend([review.text for review in review_elements])

            driver.back()  # 이전 페이지로 돌아가기

        # 다음 페이지가 있는지 확인하고 이동
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, 'a.next')
            next_button.click()
        except:
            break  # 다음 페이지가 없으면 종료

In [ ]:
# 모든 제품군에 대해 리뷰 데이터를 수집
for category, url in urls.items():
    scrape_reviews(url, category)

In [ ]:
# 브라우저 종료
driver.quit()

In [ ]:
# 수집된 리뷰 데이터를 출력
for category, review_list in reviews.items():
    print(f"{category} 리뷰:")
    for review in review_list:
        print(f"- {review}")